In [ ]:
import os
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms

In [ ]:
os.getcwd()

In [ ]:
import torch
import torch.nn.functional as F

from torch import nn

 

class VggFace(torch.nn.Module):
    def __init__(self, classes=2622):
        """VGGFace model.
        Face recognition network.  It takes as input a Bx3x224x224
        batch of face images and gives as output a BxC score vector
        (C is the number of identities).
        Input images need to be scaled in the 0-1 range and then 
        normalized with respect to the mean RGB used during training.
        Args:
            classes (int): number of identities recognized by the
            network
        """
        super().__init__()
        self.conv1 = _ConvBlock(3, 32, 32)
        self.conv2 = _ConvBlock(32, 64, 64)
        self.conv3 = _ConvBlock(64, 128, 128, 128)
        self.conv4 = _ConvBlock(128, 256, 256, 256)
        self.conv5 = _ConvBlock(256, 512, 512, 512)
        self.dropout = torch.nn.Dropout(0.5)
        self.fc1 = torch.nn.Linear(7 * 7 * 512, 4096)
        self.fc2 = torch.nn.Linear(4096, 4096)
        self.fc3 = torch.nn.Linear(4096, classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x


class _ConvBlock(torch.nn.Module):
    """A Convolutional block."""

    def __init__(self, *units):
        """Create a block with len(units) - 1 convolutions.
        
        convolution number i transforms the number of channels from 
        units[i - 1] to units[i] channels.
        """
        super().__init__()
        self.convs = torch.nn.ModuleList([
            torch.nn.Conv2d(in_, out, 3, 1, 1)
            for in_, out in zip(units[:-1], units[1:])
        ])
        
    def forward(self, x):
        # Each convolution is followed by a ReLU, then the block is
        # concluded by a max pooling.
        for c in self.convs:
            x = F.relu(c(x))
        return F.max_pool2d(x, 2, 2, 0, ceil_mode=True)

In [ ]:
import torch

#패스 지정
dataset_path = "/Users/Administrator/Desktop/facebank"
model_weight_save_path = "/Users/Administrator/Desktop/pytorch/save/vgg"
num_classes = 400

batch_size = 25
num_workers = 2
lr = 1e-2

total_epoch = 100

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
import torchvision.datasets as datasets
import os


# 훈련 및 테스트데이터 로드
traindir = os.path.join(dataset_path, '/Users/Administrator/Desktop/facebank/train')
testdir = os.path.join(dataset_path, '/Users/Administrator/Desktop/facebank/train')

# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #normalize,
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=num_workers, pin_memory=True, drop_last=False)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(testdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        #normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=num_workers, pin_memory=True)

In [ ]:
model = VggFace(num_classes).to(device)

In [ ]:
print(num_classes)

In [ ]:
CEloss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
import numpy as np
import torch

total_iteration_per_epoch = int(np.ceil(len(train_dataset)/batch_size))

for epoch in range(1, total_epoch + 1):
    model.train()
    for itereation, (input, target) in enumerate(train_loader):
        images = input.to(device)
        labels = target.to(device)

        # Forward pass
        outputs = model(images)
        loss = CEloss(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch [{}/{}], Iteration [{}/{}] Loss: {:.4f}'.format(epoch, total_epoch, itereation+1, total_iteration_per_epoch, loss.item()))
    if epoch % 5 == 0:
        torch.save(model.state_dict(), model_weight_save_path + 'model.pt', _use_new_zipfile_serialization=False)
        #torch.save(model, model_weight_save_path + 'model.pt') #전체 모델저장
        #torch.save(model.state_dict(), model_weight_save_path + 'model' + str(epoch) + ".pth") #모델 객체의 static 저장
        #torch.save({'model': model.state_dict(),'optimizer': optimizer.state_dict()}, PATH + 'all.tar') #학습중 진행상황 등 모든 전체 정보저장
        
        
    
    model.eval()
    with torch.no_grad():
      correct = 0
      total = 0
      for input, target in test_loader:
          images = input.to(device)
          labels = target.to(device)

          # Forward pass
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += len(labels)
          correct += (predicted == labels).sum().item()

      print('Epoch [{}/{}], Test Accuracy of the model on the {} test images: {} %'.format(epoch, total_epoch, total, 100 * correct / total))